# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk
import sqlite3
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_response', con=engine)

X = df.filter(items=['id', 'message', 'original', 'genre'])
y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'],  axis=1).astype(float)
y['related']=y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(x).lower().strip() for x in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'],)

In [7]:
col_names = y.columns.tolist()
print(classification_report(y_train, y_pred_train, target_names= col_names))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00     15016
               request       1.00      0.92      0.96      3375
                 offer       1.00      0.64      0.78        88
           aid_related       1.00      0.96      0.98      8102
          medical_help       1.00      0.82      0.90      1529
      medical_products       1.00      0.81      0.90       962
     search_and_rescue       1.00      0.74      0.85       553
              security       1.00      0.67      0.80       345
              military       1.00      0.81      0.90       617
                 water       1.00      0.90      0.95      1247
                  food       1.00      0.92      0.96      2192
               shelter       1.00      0.90      0.95      1724
              clothing       1.00      0.79      0.88       292
                 money       1.00      0.76      0.86       455
        missing_people       1.00      

In [8]:
print(classification_report(y_test, y_pred_test, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.85      0.90      0.87      5078
               request       0.82      0.40      0.54      1099
                 offer       0.00      0.00      0.00        30
           aid_related       0.78      0.43      0.56      2758
          medical_help       0.69      0.02      0.04       555
      medical_products       0.74      0.04      0.08       351
     search_and_rescue       0.50      0.01      0.01       171
              security       1.00      0.01      0.02       126
              military       0.67      0.02      0.03       243
                 water       0.89      0.12      0.21       425
                  food       0.84      0.26      0.39       731
               shelter       0.85      0.12      0.21       590
              clothing       0.75      0.03      0.05       113
                 money       0.67      0.01      0.03       149
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters =  {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 92.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 4], 'clf__min_samples_split': [2, 5, 10], 'clf__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred_testing = cv.predict(X_test['message'])
y_pred_training = cv.predict(X_train['message'])

In [11]:
print(classification_report(y_test, y_pred_testing, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5078
               request       0.88      0.45      0.60      1099
                 offer       0.00      0.00      0.00        30
           aid_related       0.82      0.51      0.63      2758
          medical_help       0.75      0.01      0.01       555
      medical_products       0.69      0.03      0.05       351
     search_and_rescue       1.00      0.01      0.01       171
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       243
                 water       0.92      0.13      0.23       425
                  food       0.91      0.29      0.44       731
               shelter       0.89      0.08      0.15       590
              clothing       0.67      0.02      0.03       113
                 money       1.00      0.03      0.05       149
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
print(classification_report(y_train.values, y_pred_training, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00     15016
               request       1.00      0.94      0.97      3375
                 offer       1.00      0.33      0.50        88
           aid_related       1.00      0.99      1.00      8102
          medical_help       1.00      0.80      0.89      1529
      medical_products       1.00      0.77      0.87       962
     search_and_rescue       1.00      0.60      0.75       553
              security       1.00      0.55      0.71       345
              military       1.00      0.80      0.89       617
                 water       1.00      0.92      0.96      1247
                  food       1.00      0.94      0.97      2192
               shelter       1.00      0.90      0.95      1724
              clothing       1.00      0.73      0.84       292
                 money       1.00      0.65      0.79       455
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 5,
 'clf__n_estimators': 40}

### 9. Export your model as a pickle file

In [14]:
s = pickle.dumps('clf')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.